## Code for Guassian Regression Process

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, DotProduct, ExpSineSquared, Matern, RBF, RationalQuadratic, WhiteKernel

from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm

### Dataset

In [ ]:
data = f"dataset/dataset_City.csv"
df = pd.read_csv(data)

In [ ]:
data = np.array(df.cum_casos.values)
data = data.reshape(-1, 1)

CRdata = data/1000
CRdata  = np.ravel(CRdata)

In [ ]:
plt.figure(1, figsize=(11, 7), dpi=80, facecolor='w', edgecolor='k')
plt.plot(data, linewidth = 3)
plt.legend(['Time series'])
plt.grid(color='k', linestyle='--', linewidth=0.1)
plt.ylabel('X', fontsize = 14)
plt.xlabel('Y', fontsize = 14)
plt.title('Data', fontsize = 14)

In [ ]:
trainL = len(CRdata) - 60
t = np.linspace(1,len(CRdata),len(CRdata))
t = t.reshape(len(t),1)
t = np.atleast_2d(t)

t_tr  = t[0:trainL]
t_test = t[trainL:]

CR_tr = CRdata[0:trainL]
CR_test = CRdata[trainL:]

In [ ]:
def half_data(data, time):
    half_data = []
    half_time = []
    for i,item in enumerate(data):
        if i%2 == 0:
            half_data.append(item)
            half_time.append(time[i])

    return np.array(half_data), np.array(half_time)

### GPR Model

In [ ]:
def GPR_Model(CR_tr, t_tr, CR_test, t_test):
    t = np.append(t_tr, t_test).reshape(-1, 1)
    CRdata = np.append(CR_tr, CR_test)

    kernel = RBF() # Kernel combinations

    model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=50, alpha=10, normalize_y=False)
    model.fit(t_tr, CR_tr)

    CRpred_tr, sigma_tr = model.predict(t_tr, return_std=True)
    CRpred_test, sigma_test = model.predict(t_test, return_std=True)
    CRpred, sigma = model.predict(t, return_std=True)

    r2_tr = r2_score(CR_tr, CRpred_tr)
    r2_test = r2_score(CR_test, CRpred_test)
    r2_combined = r2_score(CRdata, CRpred)

    mse_combined = mean_squared_error(CRdata, CRpred)
    std_combined = np.std(CRpred - CRdata)

    return {
        'CRpred_tr': CRpred_tr,
        'sigma_tr': sigma_tr,
        'CRpred_test': CRpred_test,
        'sigma_test': sigma_test,
        'CRpred': CRpred,
        'sigma': sigma,
        't': t,
        'CRdata': CRdata,
        'mse': mse_combined,
        'std': std_combined,
        'r2_tr': r2_tr,
        'r2_combined': r2_combined,
        'r2_test': r2_test,
        'kernel': model.kernel_
    }

### Evaluation metrics

In [ ]:
def find_best_model(CR_tr, t_tr, CR_test, t_test, patience=10):
    best_result = None
    no_improvement_count = 0

    with tqdm(total=patience, desc="Progress") as pbar:
        while no_improvement_count < patience:
            result = GPR_Model(CR_tr, t_tr, CR_test, t_test)

            if best_result is None or result['mse'] < best_result['mse']:
                best_result = result
                no_improvement_count = 0
            else:
                no_improvement_count += 1
                pbar.update(1)

    return best_result

In [ ]:
best_result = find_best_model(CR_tr, t_tr, CR_test, t_test)

In [ ]:
print("Best learned kernel:", best_result['kernel'])
print("Best MSE:", best_result['mse'])
print("Best STD:", best_result['std'])
print("Best R² train:", best_result['r2_tr'])
print("Best R²:", best_result['r2_combined'])
print("Best R² test:", best_result['r2_test'])

### Graph

In [ ]:
def plot_gpr(CRdata, CRpred, sigma, t, t_test):
  plt.figure(1, figsize=(12, 7), dpi=80, facecolor='w', edgecolor='k')
  mpl.style.use('default')

  plt.plot(t, CRdata, 'ko', markersize=3, mfc='none', linewidth=1, label='Data')
  plt.axvline(x=t_test[0], linestyle='--', color='black')
  plt.plot(t, CRpred, 'b-', label='Prediction via GPR model', linewidth=3)
  plt.fill(np.concatenate([t, t[::-1]]),
          np.concatenate([CRpred - 1.9600 * sigma,
                          (CRpred + 1.9600 * sigma)[::-1]]),
          alpha=.25, fc='b', ec='None', label='confidence interval of 95%')
  plt.legend(loc='upper left', fontsize=14)
  plt.grid(color='k', linestyle='--', linewidth=0.1)
  plt.ylabel('Total cases (in thousands)', fontsize=14)
  plt.xlabel('Day', fontsize=14)
  plt.title('Data Chart - GPR Model', fontsize=14)
  plt.gca().spines['top'].set_visible(False)
  plt.gca().spines['right'].set_visible(False)

  plt.show()

In [ ]:
plot_gpr(best_result['CRdata'], best_result['CRpred'], best_result['sigma'], best_result['t'], t_test)